In [1]:
import os
import re
from datetime import datetime
from io import StringIO
from pathlib import Path

import pandas as pd

In [2]:
metadata_file = "downloads.tsv"

data_dir = Path("./data/sdmbench")
result_dir = Path("./results/spatial_embedding")

seed = 42

In [3]:
result_dir.mkdir(exist_ok=True, parents=True)

In [4]:
metadata = pd.read_table(metadata_file, index_col=0)

metadata = metadata.assign(path=lambda df: data_dir / df["url"].str.split("/").str[-1])

# Settings

In [5]:
n_pcs = 30
n_genes = 3_000

In [6]:
conda_env = "spatialleiden"
conda_path = "~/miniconda3/bin/activate"

log_dir = result_dir / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

conda_cmd = f"source {conda_path} {conda_env}"

path = Path(os.getcwd())

# MERFISH

In [10]:
sample = metadata.loc["MERFISH_0.14"]

sample_dir = result_dir / sample.name
sample_dir.mkdir(exist_ok=True, parents=True)

for neighbors in ["delaunay", 5, 10, 15, 20, 50]:
    for w in [0.5, 1, 1.5, 2, 4]:
        name = f"neighbors_{neighbors}_w{w:.1f}"
        out = sample_dir / f"{name}.tsv"
        cmd = (
            f"{path/'benchmark.py'} "
            f"-i {sample.path} "
            f"-o {out} "
            f"--spatial_weight {w} "
            f"--n_pcs {n_pcs} "
            f"--neighbors {neighbors} "
            f"--seed {seed} "
            "--log "
        )

        _ = os.popen(
            f"sbatch -J {name} --mem=5G -n 8 -N 1 "
            f"-o {log_dir/name}.txt "
            f'--wrap="{conda_cmd} && {cmd}" '
        ).read()

# Stereo-seq

In [11]:
sample = metadata.loc["E10.5_E1S2.MOSTA"]

sample_dir = result_dir / sample.name
sample_dir.mkdir(exist_ok=True, parents=True)

for n_ring in [1, 2]:
    for n_neigh in [4, 8]:
        for w in [0.25, 0.5, 1, 1.5, 2]:
            name = f"neighbors_{n_ring}x{n_neigh}_w{w:.1f}"
            out = sample_dir / f"{name}.tsv"
            cmd = (
                f"{path/'benchmark.py'} "
                f"-i {sample.path} "
                f"-o {out} "
                f"--spatial_weight {w} "
                f"--n_pcs {n_pcs} "
                f"--n_genes {n_genes} "
                f"--seed {seed} "
                f"--n_rings {n_ring} "
                f"--neighbors {n_neigh} "
                "--stereoseq "
            )

            _ = os.popen(
                f"sbatch -J {name} --mem=5G -n 8 -N 1 "
                f"-o {log_dir/name}.txt "
                f'--wrap="{conda_cmd} && {cmd}" '
            ).read()